In [1]:
import pandas as pd
import numpy as np
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.utils import resample
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import math 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

from sklearn.utils import class_weight

%matplotlib inline 
sns.set(color_codes=True)

In [2]:
def binary(df, min):
  for i in min: 
    df.loc[df.REASON == i, 'REASON'] = 0
  df.loc[df.REASON != 0, 'REASON'] = 1
  return df

In [3]:
data = pd.read_csv('training_data.csv')

In [4]:
data = data[data.REASON != 7]
data = data.loc[:, data.columns != 'CASEID']
data= data.loc[:, data.columns != 'Unnamed: 0']
data= data.loc[:, data.columns != 'ADMYR']

In [5]:
# Labels are the values we want to predict
labels = np.array(data['REASON'])
# Remove the labels from the features
# axis 1 refers to the columns
features= data.drop('REASON', axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=0)

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf.fit(X_train, y_train);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
# Calculate the absolute errors


print("Accuracy:", metrics.accuracy_score(y_test, predictions))

In [ ]:
# Create a confusion matrix
cnf_matrix = confusion_matrix(y_test, predictions)

# Create heatmap from the confusion matrix
%matplotlib inline
class_names=[False, True] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
#model for treatment effective/not effective
data_eff =  data[data['REASON'].isin([1,2])] 
data_eff['REASON'].value_counts()

In [ ]:
train_eff, test_eff = train_test_split(data_eff, test_size=0.2, shuffle=True)

train_eff['REASON'].value_counts()

In [ ]:
df_majority = train_eff[train_eff.REASON==1]
df_minority = train_eff[train_eff.REASON==2]
 
# Upsample minority class
df_min_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=115835,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_resampled_eff = pd.concat([df_min_upsampled, df_majority])
 
# Display new class counts
df_resampled_eff.REASON.value_counts()

In [ ]:
# Labels are the values we want to predict
y_train_eff = np.array(df_resampled_eff['REASON'])
# Remove the labels from the features
# axis 1 refers to the columns
X_train_eff = df_resampled_eff.drop('REASON', axis = 1)
# Saving feature names for later use
feature_list_train_eff = list(X_train_eff.columns)
# Convert to numpy array
X_train_eff = np.array(X_train_eff)

In [ ]:
# Labels are the values we want to predict
y_test_eff = np.array(test_eff['REASON'])
# Remove the labels from the features
# axis 1 refers to the columns
X_test_eff = test_eff.drop('REASON', axis = 1)
# Saving feature names for later use
feature_list_train_eff = list(X_test_eff.columns)
# Convert to numpy array
X_test_eff = np.array(X_test_eff)

In [ ]:
model_res = rf.fit(X_train_eff, y_train_eff)

In [ ]:
y_pred_eff = model_res.predict(X_test_eff)


print("Accuracy:", metrics.accuracy_score(y_test_eff, y_pred_eff))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
cm = metrics.confusion_matrix(y_test_eff, y_pred_eff)
disc = ConfusionMatrixDisplay(cm)
disc.plot()
print(cm)

In [ ]:
nd = data.copy(deep=True)

In [ ]:
binary(nd,[6])

In [ ]:
train, test = train_test_split(nd, test_size=0.2, shuffle=True)

In [ ]:
df_majority = train[train.REASON==1]
df_minority = train[train.REASON==0]
 
# Upsample minority class
df_maj_downsampled = resample(df_majority, 
                                 replace=False,     # sample with replacement
                                 n_samples=1000,    # to match majority class
                                 random_state=123) # reproducible results
# Upsample minority class
df_min_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=1000,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_resampled = pd.concat([df_min_upsampled, df_maj_downsampled])
 
# Display new class counts
df_resampled.REASON.value_counts()

In [ ]:
# Labels are the values we want to predict
y_train_nd = np.array(df_resampled['REASON'])
# Remove the labels from the features
# axis 1 refers to the columns
X_train_nd = df_resampled.drop('REASON', axis = 1)
# Saving feature names for later use
feature_list_train_nd = list(X_train_nd.columns)
# Convert to numpy array
X_train_nd = np.array(X_train_nd)

In [ ]:
# Labels are the values we want to predict
y_test_nd = np.array(test['REASON'])
# Remove the labels from the features
# axis 1 refers to the columns
X_test_nd = test.drop('REASON', axis = 1)
# Saving feature names for later use
feature_list_train_nd = list(X_test_nd.columns)
# Convert to numpy array
X_test_nd = np.array(X_test_nd)

In [ ]:
# Define Parameters
max_depth=[2, 8, 16]
n_estimators = [64, 128, 256]
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)

# Build the grid search
dfrst = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
grid = GridSearchCV(estimator=dfrst, param_grid=param_grid, cv = 5)
grid_results = grid.fit(X_train_nd, y_train_nd)

# Summarize the results in a readable format
print("Best: {0}, using {1}".format(grid_results.cv_results_['mean_test_score'], grid_results.best_params_))
results_df = pd.DataFrame(grid_results.cv_results_)
results_df

In [ ]:
rfb = RandomForestClassifier(n_estimators=128, max_depth=16)

rfb.fit(X_train_nd, y_train_nd);

In [ ]:
y_pred_nd=rfb.predict(X_test_nd)

print("Accuracy:", metrics.accuracy_score(y_test_nd, y_pred_nd))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
cm = metrics.confusion_matrix(y_test_nd, y_pred_nd)
disc = ConfusionMatrixDisplay(cm)
disc.plot()
print(cm)

In [ ]:
#treatment completed vs not
tc = data.copy(deep=True)

In [ ]:
binary(tc,[2])

In [ ]:
train_tc, test_tc = train_test_split(tc, test_size=0.2, shuffle=True)

train_tc['REASON'].value_counts()

In [ ]:
df_majority = train_tc[train_tc.REASON==1]
df_minority = train_tc[train_tc.REASON==0]
 
# Upsample minority class
df_min_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=195180,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_resampled_tc = pd.concat([df_min_upsampled, df_majority])
 
# Display new class counts
df_resampled_tc.REASON.value_counts()

In [ ]:
# Labels are the values we want to predict
y_train_tc = np.array(df_resampled_tc['REASON'])
# Remove the labels from the features
# axis 1 refers to the columns
X_train_tc = df_resampled_tc.drop('REASON', axis = 1)
# Saving feature names for later use
feature_list_train_nd = list(X_train_tc.columns)
# Convert to numpy array
X_train_tc = np.array(X_train_tc)

In [ ]:
# Labels are the values we want to predict
y_test_tc = np.array(test_tc['REASON'])
# Remove the labels from the features
# axis 1 refers to the columns
X_test_tc = test_tc.drop('REASON', axis = 1)
# Saving feature names for later use
feature_list_train_tc = list(X_test_tc.columns)
# Convert to numpy array
X_test_tc= np.array(X_test_tc)

In [ ]:
rf.fit(X_train_tc, y_train_tc);

In [ ]:
rf.fit(X_train_tc, y_train_tc);

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
cm = metrics.confusion_matrix(y_test_tc, y_pred_tc)
disc = ConfusionMatrixDisplay(cm)
disc.plot()
print(cm)

In [ ]:
import pandas as pd

tcc = tc.loc[:, tc.columns != 'REASON']


features = tcc.columns
importances = rf.feature_importances_
indices = np.argsort(importances)

# customized number 
num_features = 10 

plt.figure(figsize=(10,100))
plt.title('Feature Importances')

# only plot the customized number of features
plt.barh(range(num_features), importances[indices[-num_features:]], color='b', align='center')
plt.yticks(range(num_features), [features[i] for i in indices[-num_features:]])
plt.xlabel('Relative Importance')
plt.show()